In [1]:
import pandas as pd
years = [18, 19, 20, 21, 22, 23, 24, 25]
headers = [
    "transaction_id",
    "price",
    "date_of_transfer",
    "postcode",
    "property_type",
    "new_build_flag",
    "tenure_type",
    "primary_addressable_object_name",
    "secondary_addressable_object_name",
    "street",
    "locality",
    "town_city",
    "district",
    "county",
    "ppd_category_type",
    "record_status"
]
drop = [
    "transaction_id",
    "primary_addressable_object_name",
    "secondary_addressable_object_name",
    "street",
    "locality",
    "ppd_category_type",
    "record_status"
]
dfs = {}
for year in years:
    url = f"http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-20{year}.csv"
    df = pd.read_csv(url)
    df.columns = headers
    df = df.drop(columns=drop)
    dfs[f"df20{year}"] = df
    print(f"{year} done")
dfs

18 done
19 done
20 done
21 done
22 done


C:\Users\lenovo\AppData\Local\Temp\ipykernel_18140\2815900718.py:33: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


23 done
24 done
25 done


{'df2018':           price  date_of_transfer  postcode property_type new_build_flag  \
 0        253500  2018-09-24 00:00    M6 8GQ             D              N   
 1        231950  2018-09-28 00:00   WA3 2UE             D              Y   
 2        112500  2018-08-29 00:00   OL6 6RJ             S              N   
 3        184995  2018-06-15 00:00   M46 0TW             S              Y   
 4        214995  2018-09-28 00:00   M28 3XS             D              Y   
 ...         ...               ...       ...           ...            ...   
 1037248  100000  2018-11-16 00:00   DH4 7HW             T              N   
 1037249  334950  2018-11-30 00:00  NE31 1BF             D              Y   
 1037250  106010  2018-12-12 00:00   NE2 1NY             F              N   
 1037251  144995  2018-07-26 00:00  NE27 0JA             T              Y   
 1037252  334950  2018-07-13 00:00  NE15 9BW             D              Y   
 
         tenure_type            town_city             district  

In [2]:
df_full = pd.concat(dfs)

In [3]:
df_full['Price (Thousands)'] = df_full['price'] / 1000
df_full.drop(columns=['price'], inplace=True)
df_full.describe()

,Price (Thousands)
count,7.219742e+06
mean,3.818475e+02
std,1.711452e+03
min,1.000000e-03
25%,1.650000e+02
50%,2.600000e+02
75%,4.000000e+02
max,9.000000e+05


In [4]:
df_full.isna().sum()

date_of_transfer         0
postcode             24503
property_type            0
new_build_flag           0
tenure_type              0
town_city                0
district                 0
county                   0
Price (Thousands)        0
dtype: int64

In [5]:
df_clean = df_full.dropna(axis=0)
df_clean.describe()

,Price (Thousands)
count,7.195239e+06
mean,3.794882e+02
std,1.660432e+03
min,1.000000e-03
25%,1.650000e+02
50%,2.600000e+02
75%,4.000000e+02
max,9.000000e+05


In [6]:
df_clean.head()

date_of_transfer postcode property_type new_build_flag tenure_type  \
df2018 0  2018-09-24 00:00   M6 8GQ             D              N           F   
       1  2018-09-28 00:00  WA3 2UE             D              Y           F   
       2  2018-08-29 00:00  OL6 6RJ             S              N           F   
       3  2018-06-15 00:00  M46 0TW             S              Y           F   
       4  2018-09-28 00:00  M28 3XS             D              Y           L   

                  town_city  district              county  Price (Thousands)  
df2018 0            SALFORD   SALFORD  GREATER MANCHESTER            253.500  
       1         WARRINGTON     WIGAN  GREATER MANCHESTER            231.950  
       2  ASHTON-UNDER-LYNE  TAMESIDE  GREATER MANCHESTER            112.500  
       3         MANCHESTER     WIGAN  GREATER MANCHESTER            184.995  
       4         MANCHESTER   SALFORD  GREATER MANCHESTER            214.995

In [7]:
df_binary = pd.get_dummies(df_clean, dtype=int, columns=['tenure_type','new_build_flag', 'property_type'])
df_binary.rename(columns={'date_of_transfer':'Transfer Date','town_city':'Town/City','district':'District','county':'County','tenure_type_F':'Freehold Tenure','tenure_type_L':'Leasehold Tenure','new_build_flag_N':'Old Build','new_build_flag_Y':'New Build','property_type_D':'Detached', 'property_type_F':'Flat', 'property_type_O':'Other Property Type','property_type_S':'Semi-detached','property_type_T':'Terraced', 'postcode':'Postcode'}, inplace=True)
df_binary.head()

Transfer Date Postcode          Town/City  District  \
df2018 0  2018-09-24 00:00   M6 8GQ            SALFORD   SALFORD   
       1  2018-09-28 00:00  WA3 2UE         WARRINGTON     WIGAN   
       2  2018-08-29 00:00  OL6 6RJ  ASHTON-UNDER-LYNE  TAMESIDE   
       3  2018-06-15 00:00  M46 0TW         MANCHESTER     WIGAN   
       4  2018-09-28 00:00  M28 3XS         MANCHESTER   SALFORD   

                      County  Price (Thousands)  Freehold Tenure  \
df2018 0  GREATER MANCHESTER            253.500                1   
       1  GREATER MANCHESTER            231.950                1   
       2  GREATER MANCHESTER            112.500                1   
       3  GREATER MANCHESTER            184.995                1   
       4  GREATER MANCHESTER            214.995                0   

          Leasehold Tenure  Old Build  New Build  Detached  Flat  \
df2018 0                 0          1          0         1     0   
       1                 0          0          1         1     0   
       2                 0          1          0         0     0   
       3                 0          0          1         0     0   
       4                 1          0          1         1     0   

          Other Property Type  Semi-detached  Terraced  
df2018 0                    0              0         0  
       1                    0              0         0  
       2                    0              1         0  
       3                    0              1         0  
       4                    0              0         0

In [8]:
df_binary['Transfer Date'] = pd.to_datetime(df_binary['Transfer Date'], errors='coerce')
df_binary['Year'] = df_binary['Transfer Date'].dt.year
df_binary['Month'] = df_binary['Transfer Date'].dt.month
df_binary['Quarter'] = df_binary['Transfer Date'].dt.quarter
df_binary['Day of the Week'] = df_binary['Transfer Date'].dt.dayofweek
df_binary['Transfer Date'] = df_binary['Transfer Date'].astype(str).str[:10]
df_binary.tail()

Transfer Date  Postcode    Town/City                District  \
df2025 252142    2025-01-17   NN4 8SA  NORTHAMPTON   WEST NORTHAMPTONSHIRE   
       252143    2025-01-24  NN16 0SE    KETTERING  NORTH NORTHAMPTONSHIRE   
       252144    2025-01-14  NN10 0NB      RUSHDEN  NORTH NORTHAMPTONSHIRE   
       252145    2025-01-10   NN4 8TE  NORTHAMPTON   WEST NORTHAMPTONSHIRE   
       252146    2025-01-27  NN15 5YL    KETTERING  NORTH NORTHAMPTONSHIRE   

                               County  Price (Thousands)  Freehold Tenure  \
df2025 252142   WEST NORTHAMPTONSHIRE              98.00                0   
       252143  NORTH NORTHAMPTONSHIRE             120.00                0   
       252144  NORTH NORTHAMPTONSHIRE             604.50                1   
       252145   WEST NORTHAMPTONSHIRE             133.65                0   
       252146  NORTH NORTHAMPTONSHIRE             332.00                1   

               Leasehold Tenure  Old Build  New Build  Detached  Flat  \
df2025 252142                 1          1          0         0     1   
       252143                 1          1          0         0     1   
       252144                 0          1          0         0     0   
       252145                 1          1          0         0     1   
       252146                 0          1          0         1     0   

               Other Property Type  Semi-detached  Terraced  Year  Month  \
df2025 252142                    0              0         0  2025      1   
       252143                    0              0         0  2025      1   
       252144                    1              0         0  2025      1   
       252145                    0              0         0  2025      1   
       252146                    0              0         0  2025      1   

               Quarter  Day of the Week  
df2025 252142        1                4  
       252143        1                4  
       252144        1                1  
       252145        1                4  
       252146        1                0

In [9]:
df_binary.shape

(7195239, 19)

In [10]:
print(df_binary.dtypes)
print(df_binary.nunique())

Transfer Date           object
Postcode                object
Town/City               object
District                object
County                  object
Price (Thousands)      float64
Freehold Tenure          int64
Leasehold Tenure         int64
Old Build                int64
New Build                int64
Detached                 int64
Flat                     int64
Other Property Type      int64
Semi-detached            int64
Terraced                 int64
Year                     int32
Month                    int32
Quarter                  int32
Day of the Week          int32
dtype: object
Transfer Date             2731
Postcode               1162297
Town/City                 1150
District                   361
County                     117
Price (Thousands)       143229
Freehold Tenure              2
Leasehold Tenure             2
Old Build                    2
New Build                    2
Detached                     2
Flat                         2
Other Property Type      

In [11]:
df_binary['Postcode Area'] = df_binary['Postcode'].str.extract(r'^([A-Z]+)')
df_binary['Postcode District'] = df_binary['Postcode'].str.extract(r'^([A-Z]+[0-9]+)')
df_binary.head()

Transfer Date Postcode          Town/City  District  \
df2018 0    2018-09-24   M6 8GQ            SALFORD   SALFORD   
       1    2018-09-28  WA3 2UE         WARRINGTON     WIGAN   
       2    2018-08-29  OL6 6RJ  ASHTON-UNDER-LYNE  TAMESIDE   
       3    2018-06-15  M46 0TW         MANCHESTER     WIGAN   
       4    2018-09-28  M28 3XS         MANCHESTER   SALFORD   

                      County  Price (Thousands)  Freehold Tenure  \
df2018 0  GREATER MANCHESTER            253.500                1   
       1  GREATER MANCHESTER            231.950                1   
       2  GREATER MANCHESTER            112.500                1   
       3  GREATER MANCHESTER            184.995                1   
       4  GREATER MANCHESTER            214.995                0   

          Leasehold Tenure  Old Build  New Build  ...  Flat  \
df2018 0                 0          1          0  ...     0   
       1                 0          0          1  ...     0   
       2                 0          1          0  ...     0   
       3                 0          0          1  ...     0   
       4                 1          0          1  ...     0   

          Other Property Type  Semi-detached  Terraced  Year  Month  Quarter  \
df2018 0                    0              0         0  2018      9        3   
       1                    0              0         0  2018      9        3   
       2                    0              1         0  2018      8        3   
       3                    0              1         0  2018      6        2   
       4                    0              0         0  2018      9        3   

          Day of the Week  Postcode Area Postcode District  
df2018 0                0              M                M6  
       1                4             WA               WA3  
       2                2             OL               OL6  
       3                4              M               M46  
       4                4              M               M28  

[5 rows x 21 columns]

In [12]:
df_binary.to_csv('Pre_Outlier_removal.csv', index=False)

In [13]:
lower = df_binary['Price (Thousands)'].quantile(0.005)
upper = df_binary['Price (Thousands)'].quantile(0.995)
df_no_outliers = df_binary[(df_binary['Price (Thousands)'] >= lower) & (df_binary['Price (Thousands)'] <= upper)]
df_no_outliers.describe()

,Price (Thousands),Freehold Tenure,Leasehold Tenure,Old Build,New Build,Detached,Flat,Other Property Type,Semi-detached,Terraced,Year,Month,Quarter,Day of the Week
count,7.124208e+06,7.124208e+06,7.124208e+06,7.124208e+06,7.124208e+06,7.124208e+06,7.124208e+06,7.124208e+06,7.124208e+06,7.124208e+06,7.124208e+06,7.124208e+06,7.124208e+06,7.124208e+06
mean,3.298715e+02,7.683223e-01,2.316777e-01,8.882258e-01,1.117742e-01,2.339497e-01,1.779576e-01,4.700200e-02,2.681612e-01,2.729296e-01,2.021035e+03,6.576128e+00,2.504003e+00,2.694955e+00
std,2.788689e+02,4.219042e-01,4.219042e-01,3.150885e-01,3.150885e-01,4.233406e-01,3.824771e-01,2.116431e-01,4.430020e-01,4.454650e-01,2.044985e+00,3.433242e+00,1.124971e+00,1.466922e+00
min,1.800000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.018000e+03,1.000000e+00,1.000000e+00,0.000000e+00
25%,1.650000e+02,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.019000e+03,3.000000e+00,1.000000e+00,2.000000e+00
50%,2.600000e+02,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.021000e+03,7.000000e+00,3.000000e+00,3.000000e+00
75%,3.999950e+02,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,2.023000e+03,1.000000e+01,4.000000e+00,4.000000e+00
max,3.000000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.025000e+03,1.200000e+01,4.000000e+00,6.000000e+00


In [14]:
df_no_outliers.to_csv('ETL_Final.csv', index=False)